In [39]:
file_path = "/home/ali47/Desktop/project_2/Finance_Complaint/finance_artifact/data_validation/20230207_165129/accepted_data/finance_complaint"

In [2]:
from finance_complaint.config.spark_manager import spark_session

bash: /home/ali47/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/ali47/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
23/02/09 02:20:19 WARN Utils: Your hostname, md-lenovo resolves to a loopback address: 127.0.1.1; using 192.168.29.135 instead (on interface wlp2s0)
23/02/09 02:20:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ali47/Desktop/project_2/Finance_Complaint/venv_project2/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ali47/.ivy2/cache
The jars for the packages stored in: /home/ali47/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2703b1df-9388-4fe8-bc77-77d51bc92963;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.7.4 in central
	found commons-logging#commons-logging;1.1.1 in central
	found org.apache.httpcomponents#httpclient;4.2 in central
	found org.apache.httpcomponents#httpcore;4.2 in central
	found commons-codec#commons-codec;1.3 in central
	found com.fasterxml.jackson.core#jackson-core;2.1.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.1.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.1.1 in central
	found joda-time#joda-time;2.12.2 in central
	[2.12.2] joda-time#joda-time;[2.2,)
	found org.apache.hadoop#hadoop-aws;2.7.3 in central
	found org.apache.hadoop#hadoop-common;2.7.3 in centra

In [3]:
spark_session

In [40]:
df = spark_session.read.parquet(file_path)

In [41]:
Expected_Column = ['consumer_disputed', 'company_response', 'consumer_consent_provided', 'submitted_via', 'issue', 'date_sent_to_company', 'date_received']

In [42]:
df = df.select(*Expected_Column)

In [43]:
df.show()

+-----------------+--------------------+-------------------------+-------------+--------------------+--------------------+--------------------+
|consumer_disputed|    company_response|consumer_consent_provided|submitted_via|               issue|date_sent_to_company|       date_received|
+-----------------+--------------------+-------------------------+-------------+--------------------+--------------------+--------------------+
|              N/A|Closed with non-m...|     Consent not provided|          Web|Problem with a cr...|2022-10-11T12:00:...|2022-10-11T12:00:...|
|              N/A|Closed with non-m...|     Consent not provided|          Web|Problem with a cr...|2022-10-10T12:00:...|2022-10-10T12:00:...|
|              N/A|Closed with non-m...|         Consent provided|          Web|Problem with a cr...|2022-10-12T12:00:...|2022-10-12T12:00:...|
|              N/A|Closed with non-m...|         Consent provided|          Web|Improper use of y...|2022-10-11T12:00:...|2022-10-11T12:

In [44]:
from pyspark.ml.pipeline import Pipeline
from typing import List

In [45]:
from pyspark.ml import Transformer, Estimator, Model
from pyspark.ml.param.shared import Param, Params, TypeConverters, HasOutputCols, HasInputCols
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark import keyword_only
from pyspark.sql.types import TimestampType, LongType
from pyspark.sql.functions import col, udf, desc
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler,OneHotEncoder, Tokenizer, HashingTF, IDF, StandardScaler


In [46]:
df.printSchema()

root
 |-- consumer_disputed: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- date_received: string (nullable = true)



In [47]:
df=df.withColumn('date_received',col("date_received").cast(TimestampType()))

In [48]:
df = df.withColumn('date_sent_to_company', col('date_sent_to_company').cast(TimestampType()))

In [49]:
df = df.withColumn("diff_in_days",(col('date_sent_to_company').cast(LongType())-col('date_received').cast(LongType()))/(60*60*24))

In [50]:
df.show(5)

+-----------------+--------------------+-------------------------+-------------+--------------------+--------------------+-------------------+------------+
|consumer_disputed|    company_response|consumer_consent_provided|submitted_via|               issue|date_sent_to_company|      date_received|diff_in_days|
+-----------------+--------------------+-------------------------+-------------+--------------------+--------------------+-------------------+------------+
|              N/A|Closed with non-m...|     Consent not provided|          Web|Problem with a cr...| 2022-10-11 22:30:00|2022-10-11 22:30:00|         0.0|
|              N/A|Closed with non-m...|     Consent not provided|          Web|Problem with a cr...| 2022-10-10 22:30:00|2022-10-10 22:30:00|         0.0|
|              N/A|Closed with non-m...|         Consent provided|          Web|Problem with a cr...| 2022-10-12 22:30:00|2022-10-12 22:30:00|         0.0|
|              N/A|Closed with non-m...|         Consent provide

In [51]:
df = df.drop(*['date_sent_to_company', "date_received"])

In [52]:
from pyspark.sql import DataFrame

In [53]:
train_dataframe, test_dataframe = df.randomSplit([0.7, 0.3])

In [54]:
train_dataframe.count()

7025

In [55]:
df.printSchema()

root
 |-- consumer_disputed: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- diff_in_days: double (nullable = true)



In [56]:
numerical_features = ['diff_in_days']
one_hot_features = ['company_response', 'consumer_consent_provided', 'submitted_via']
tokenizer_feature = ['issue']
target_feature = ['company_response']
required_input_features = numerical_features+one_hot_features+tokenizer_feature
required_input_features

['diff_in_days',
 'company_response',
 'consumer_consent_provided',
 'submitted_via',
 'issue']

In [57]:
from pyspark.sql.functions import isnull
null_counts = {}
for column in required_input_features:
    null_count= df.filter(isnull(col(column))).count()
    null_counts.update({column:null_count})
null_counts

{'diff_in_days': 0,
 'company_response': 0,
 'consumer_consent_provided': 8,
 'submitted_via': 0,
 'issue': 0}

In [58]:
stages = []

# impute null in numerical columns ********* done
# frequrency imputer in one_hot_features ***** done
# string_indexer for imputed one_hot_features *** done
#  one_hot_encoder for  string_indexer of imputed one_hot_features *** done
# tokenizing the tokenizer feature
# Hashing the tokenized feature
# IDF for above tokenized feature
# vector assembler
# standard scaler
# pipeline for all the above

In [59]:
impute_num_col = Imputer(strategy='median', inputCol = "diff_in_days",  outputCol = 'im_diff_in_days' )
stages.append(impute_num_col)

In [60]:
class FrequencyImputer(
    Estimator, HasInputCols, HasOutputCols,
    DefaultParamsReadable, DefaultParamsWritable):
    topCategorys = Param(Params._dummy(), "getTopCategorys", "getTopCategorys",
                         typeConverter=TypeConverters.toListString)

    @keyword_only
    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None, ):
        super(FrequencyImputer, self).__init__()
        self.topCategorys = Param(self, "topCategorys", "")
        self._setDefault(topCategorys="")
        kwargs = self._input_kwargs
        print(kwargs)
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols: List[str] = None, outputCols: List[str] = None, ):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setTopCategorys(self, value: List[str]):
        return self._set(topCategorys=value)

    def getTopCategorys(self):
        return self.getOrDefault(self.topCategorys)

    def setInputCols(self, value: List[str]):
        """
        Sets the value of :py:attr:`inputCol`.
        """
        return self._set(inputCols=value)

    def setOutputCols(self, value: List[str]):
        """
        Sets the value of :py:attr:`outputCol`.
        """
        return self._set(outputCols=value)

    def _fit(self, dataset: DataFrame):
        inputCols = self.getInputCols()
        topCategorys = []
        for column in inputCols:
            categoryCountByDesc = dataset.groupBy(column).count().filter(f'{column} is not null').sort(
                desc('count'))
            topCat = categoryCountByDesc.take(1)[0][column]
            topCategorys.append(topCat)

        self.setTopCategorys(value=topCategorys)

        estimator = FrequencyImputerModel(inputCols=self.getInputCols(),
                                          outputCols=self.getOutputCols())

        estimator.setTopCategorys(value=topCategorys)
        return estimator


class FrequencyImputerModel(FrequencyImputer, Transformer):

    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None, ):
        super(FrequencyImputerModel, self).__init__(inputCols=inputCols, outputCols=outputCols)

    def _transform(self, dataset: DataFrame):
        topCategorys = self.getTopCategorys()
        outputCols = self.getOutputCols()

        updateMissingValue = dict(zip(outputCols, topCategorys))

        inputCols = self.getInputCols()
        for outputColumn, inputColumn in zip(outputCols, inputCols):
            dataset = dataset.withColumn(outputColumn, col(inputColumn))
            # print(dataset.columns)
            # print(outputColumn, inputColumn)

        dataset = dataset.na.fill(updateMissingValue)

        return dataset

In [61]:
frequency_imputer = FrequencyImputer(inputCols=one_hot_features, 
                                        outputCols=['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via'])

stages.append(frequency_imputer)

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [62]:
string_indexer = StringIndexer(inputCols=['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via'],
                                             outputCols=['si_im_company_response', 'si_im_consumer_consent_provided', 'si_im_submitted_via'])
                                
stages.append(string_indexer)

In [63]:
one_hot_encoder = OneHotEncoder(inputCols=string_indexer.getOutputCols(), 
                                outputCols=[f"ohe_{col}" for col in string_indexer.getOutputCols()])
                            
stages.append(one_hot_encoder)

In [64]:
tokenizer = Tokenizer(inputCol='issue', outputCol="words")

stages.append(tokenizer)

In [65]:
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=40)

stages.append(hashing_tf)

In [66]:
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol='tf_issue')

stages.append(idf)

In [67]:
vector_assembler = VectorAssembler(inputCols= one_hot_encoder.getOutputCols() + [idf.getOutputCol()] + [impute_num_col.getOutputCol()], 
                                        outputCol='va_input_features')

stages.append(vector_assembler)

In [68]:
standard_scaler = StandardScaler(inputCol=vector_assembler.getOutputCol(), 
                                    outputCol="scaled_features")
stages.append(standard_scaler)

In [69]:
pipeline = Pipeline(stages=stages)

In [70]:
train_dataframe.count()

7025

In [74]:
transformed_train_dataframe_ = pipeline.fit(train_dataframe).transform(train_dataframe)

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [75]:
transformed_train_dataframe_.show()

+-----------------+--------------------+-------------------------+-------------+--------------------+------------+---------------+--------------------+----------------------------+----------------+----------------------+-------------------------------+-------------------+--------------------------+-----------------------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|consumer_disputed|    company_response|consumer_consent_provided|submitted_via|               issue|diff_in_days|im_diff_in_days| im_company_response|im_consumer_consent_provided|im_submitted_via|si_im_company_response|si_im_consumer_consent_provided|si_im_submitted_via|ohe_si_im_company_response|ohe_si_im_consumer_consent_provided|ohe_si_im_submitted_via|               words|         rawFeatures|            tf_issue|   va_input_features|     scaled_features|
+-----------------+--------------------+-------------------------+----

In [77]:
transformed_train_dataframe_.select('scaled_features').show()

+--------------------+
|     scaled_features|
+--------------------+
|(52,[0,3,7,16,19,...|
|(52,[0,3,16,19,28...|
|(52,[0,3,11,18,26...|
|(52,[0,3,7,11,22,...|
|(52,[0,3,7,21,22,...|
|(52,[0,3,7,21,22,...|
|(52,[0,3,7,21,22,...|
|(52,[0,3,7,21,22,...|
|(52,[0,3,7,21,22,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
|(52,[0,3,7,14,16,...|
+--------------------+
only showing top 20 rows

